# Building a Spam Filter with Naive Bayes

In this project, I will use the Multinomial Naive Bayes algorithm to build a spam filter for SMS messages. My goal is to create a filter that can classify messages as spam or not with over 80% accuracy.

I will be using a dateset of 5,572 SMS messages that have already been classified by humans. The dataset is from the UCI Machine Learning Repository, and it can be found [here](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

## Exploring the Dataset

First I will read in and explore the data:

In [1]:
import pandas as pd
messages = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names = ['Label', 'SMS'])

messages.shape

(5572, 2)

In [2]:
messages.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
print(messages['Label'].value_counts())
print(messages['Label'].value_counts(normalize=True))

ham     4825
spam     747
Name: Label, dtype: int64
ham     0.865937
spam    0.134063
Name: Label, dtype: float64


The dataset has 5572 messages. 747 (~13%) are classified as spam and 4825 (~87%) are classified as ham (non-spam).

## Training and Test Set

I will need to test my filter once I have built it. To do this, I will split the dataset into two categories:
- A training set, which I'll use to "train" the computer how to classify messages.
- A test set, which I'll use to test how good the spam filter is at classifying new messages.

I will keep about 80% of the data (4458 messages) for training, and use 20% of the data (1114 messages) for testing. In order to do this I will randomize the entire datset, split it into two categories, then check to make sure each category has a similar percentage of spam messages. 

In [4]:
messages_random = messages.sample(frac=1, random_state=1)
training = messages_random.head(4458).reset_index(drop=True)
test = messages_random.tail(1114).reset_index(drop=True)

print(training.shape)
print(test.shape)
test.head()

(4458, 2)
(1114, 2)


,Label,SMS
0,ham,Later i guess. I needa do mcat study too.
1,ham,But i haf enuff space got like 4 mb...
2,spam,Had your mobile 10 mths? Update to latest Oran...
3,ham,All sounds good. Fingers . Makes it difficult ...
4,ham,"All done, all handed in. Don't know if mega sh..."


In [5]:
training['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [6]:
test['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

The training set and the test set each contain 13-14% spam messsages, which is similar to each other and similar to the original dataset.

## Data Cleaning

I need to transform the training data into a dataframe where each column represents a word in the vocabulary, and the values in each column are integers corresponding to the number of occurences of that word in the message. 

The first step is to remove all punctuation from the `SMS` column, and convert all words to lowercase.

In [7]:
training.head() # before cleaning

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [8]:
training['SMS'] = training['SMS'].str.replace('\W', ' ')
training['SMS'] = training['SMS'].str.lower()

training.head() # after cleaning

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


The next step of data cleaning is to create the vocabulary. This will be a list of every unique word in the training dataset.

In [9]:
training['SMS'] = training['SMS'].str.split()

vocabulary = []
for sentence in training['SMS']:
    for word in sentence:
        vocabulary.append(word)

print(len(vocabulary))
# Convert to a set then back to a list to get rid of duplicates
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)
print(len(vocabulary))

72427
7783


There are 7783 unique words in our training set.

The next step is to convert the vocabulary into a dictionary, where each key is a unique word from the vocabulary, and each value is a list corresponding to the number of occurences of that word in each sentence from the training data set. I will then convert this dictonary to a DataFrame, which I can concatenate with the original training set that contains the `Label` and `SMS` columns.

In [10]:
# create the dictonary with unique words
word_counts_per_sms = {}
for unique_word in vocabulary:
    word_counts_per_sms[unique_word] = [0] * len(training['SMS'])

# loop through the lists in the SMS column to populate the dictionary
for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [11]:
word_counts_per_sms_df = pd.DataFrame(word_counts_per_sms)
word_counts_per_sms_df.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [12]:
training_clean = pd.concat([training, word_counts_per_sms_df], axis=1)
training_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Constants First

I will first calculate all the terms in the Naive Bayes Algorithm that are constant - they will have the same value for every message.

These are:
- P(Spam) - the probability a message is spam
- P(Ham) - the probability a message is non-spam
- N_Spam - the number of words in all spam messages
- N_Ham - the number of words in all non-spam messages
- N_Vocabulary - the number of unique words

I will also be using Laplace smoothing, so I will create an alpha term and give it a value of 1.

In [13]:
p_spam = training_clean['Label'].value_counts(normalize=True)['spam']
p_ham = training_clean['Label'].value_counts(normalize=True)['ham']
print('P(Spam) = {}'.format(p_spam))
print('P(Ham) = {}'.format(p_ham))

P(Spam) = 0.13458950201884254
P(Ham) = 0.8654104979811574


In [14]:
ham_only = training_clean[training_clean['Label'] == 'ham']
ham_only_lengths = ham_only['SMS'].apply(len)
n_ham = ham_only_lengths.sum()

spam_only = training_clean[training_clean['Label'] == 'spam']
spam_only_lengths = spam_only['SMS'].apply(len)
n_spam = spam_only_lengths.sum()

print('n_ham = {}'.format(n_ham))
print('n_spam = {}'.format(n_spam))

n_ham = 57237
n_spam = 15190


In [15]:
n_vocabulary = len(vocabulary)
alpha = 1 # for Laplace smoothing

print('n_vocabulary = {}'.format(n_vocabulary))
print('alpha = {}'.format(alpha))

n_vocabulary = 7783
alpha = 1


## Calculating Parameters

I will now calculate P(w_i|Ham) and P(w_i|Spam) for every word in the vocabulary. I will store all of these probabilities in two dictionaries - one for Ham and one for Spam.

First I will inialize the two dictionaries where each key is a unique word (from the vocabulary) represented as a string, and every value is 0:

In [16]:
p_word_given_spam_dict = {}
p_word_given_ham_dict = {}

word_counts_per_sms = {}
for unique_word in vocabulary:
    p_word_given_spam_dict[unique_word] = 0
    p_word_given_ham_dict[unique_word] = 0

Next I will populate the dictionaries with the correct probabilities. I will loop through the vocabulary and calculate P(word|Ham) and P(word|Spam) for each word, then assign the values to the corresponding keys in the dictionaries.

In [17]:
for unique_word in vocabulary:
    n_word_given_spam = spam_only[unique_word].sum()
    p_word_given_spam = (n_word_given_spam  + alpha) / (n_spam + alpha*n_vocabulary)
    p_word_given_spam_dict[unique_word] = p_word_given_spam
    
    n_word_given_ham = ham_only[unique_word].sum()
    p_word_given_ham = (n_word_given_ham  + alpha) / (n_ham + alpha*n_vocabulary)
    p_word_given_ham_dict[unique_word] = p_word_given_ham

## Classifying A New Message

Now that I have calculated all the constants and parameters needed, I can create the spam filter. 

I will write a function that:
- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help

In [23]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in p_word_given_spam_dict:
            p_spam_given_message *= p_word_given_spam_dict[word]
            
        if word in p_word_given_ham_dict:
            p_ham_given_message *= p_word_given_ham_dict[word]
    
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

Now that I have created the function, I can test it with a few inputs:

In [24]:
classify('Hello, how are you?')

P(Spam|message): 2.251394958301714e-13
P(Ham|message): 2.3404411560421706e-10
Label: Ham


In [25]:
classify('You have won a free ipad')

P(Spam|message): 8.823034401011034e-17
P(Ham|message): 1.3305587169015532e-17
Label: Spam


In [26]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [27]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accuracy

Now I will test my spam filter on the test data. 

First I will modify my `classify` function from above to return 'spam' or 'ham' labels instead of printing them:

In [31]:
import re

def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in p_word_given_spam_dict:
            p_spam_given_message *= p_word_given_spam_dict[word]
            
        if word in p_word_given_ham_dict:
            p_ham_given_message *= p_word_given_ham_dict[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

I will then use this new function to create a new column in the test dataframe corresponding to how my spam filter classifies each SMS message:

In [32]:
test['filter_label'] = test['SMS'].apply(classify_test_set)
test.head()

,Label,SMS,filter_label
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


I will calculate the accuracy of my filter by dividing the number of correctly classified messages by the total number of classified messages.

In [38]:
test['filter_correct'] = test['Label'] == test['filter_label']
correct = test['filter_correct'].sum()
total = test['filter_correct'].count()

accuracy = correct / total
print('The accuracy of the spam filter is: {:.6%}'.format(accuracy))

The accuracy of the spam filter is: 98.743268%


In [52]:
correct_given_spam = test[test['Label'] == 'spam']['filter_correct'].sum()
total_given_spam = test[test['Label'] == 'spam']['filter_correct'].count()
accuracy_given_spam = correct_given_spam / total_given_spam

correct_given_ham = test[test['Label'] == 'ham']['filter_correct'].sum()
total_given_ham = test[test['Label'] == 'ham']['filter_correct'].count()
accuracy_given_ham = correct_given_ham / total_given_ham

print('The spam filter correctly identified {:.6%} of the spam messages'.format(accuracy_given_spam))
print('The spam filter correctly identified {:.6%} of the ham messages'.format(accuracy_given_ham))

The spam filter correctly identified 94.557823% of all spam messages
The spam filter correctly identified 99.379524% of all ham messages


Overall, the accuracy of the spam filter was very good. Of the 1114 messages in the test data set, it was able to correctly classify 98.74% of them.

The filter is particularly good at correctly identifying ham messages - it was 98.38% accurate with these messages. It was slightly less successful at correctly classifying spam messages - it was 94.56% accurate with these messages.

That being said, even 94.56% is well above my 80% goal for this project. Also, I would rather err on the side of misidentifying spam as ham, rather than misidentifying ham as spam. I think if given the choice, users would prefer to have a few spam messages get through the filter once in a while rather than have occasional messages from their friends blocked by the filter.

## Next Steps
- Isolate the 14 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions.
- Make the filtering process more complex by making the algorithm sensitive to letter case.

In [53]:
test[test['filter_correct'] == False]

,Label,SMS,filter_label,filter_correct
114,spam,Not heard from U4 a while. Call me now am here...,ham,False
135,spam,More people are dogging in your area now. Call...,ham,False
152,ham,Unlimited texts. Limited minutes.,spam,False
159,ham,26th OF JULY,spam,False
284,ham,Nokia phone is lovly..,spam,False
293,ham,A Boy loved a gal. He propsd bt she didnt mind...,needs human classification,False
302,ham,No calls..messages..missed calls,spam,False
319,ham,We have sent JD for Customer Service cum Accou...,spam,False
504,spam,Oh my god! I've found your number again! I'm s...,ham,False
546,spam,"Hi babe its Chloe, how r u? I was smashed on s...",ham,False
